In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Define the size of the synthetic dataset
num_samples = 100000  # You can adjust this number as needed

# Generate synthetic data for the features
data = {
    'duration': np.random.randint(0, 1000, size=num_samples),
    'protocol_type': np.random.choice(['tcp', 'udp', 'icmp'], size=num_samples),
    'service': np.random.choice(['http', 'ftp', 'smtp', 'dns', 'telnet'], size=num_samples),
    'flag': np.random.choice(['SF', 'REJ', 'RSTO', 'RSTOS0'], size=num_samples),
    'src_bytes': np.random.randint(0, 5000, size=num_samples),
    'dst_bytes': np.random.randint(0, 5000, size=num_samples),
    'land': np.random.choice([0, 1], size=num_samples),
    'wrong_fragment': np.random.randint(0, 100, size=num_samples),
    'urgent': np.random.randint(0, 100, size=num_samples),
    'hot': np.random.randint(0, 100, size=num_samples),
    'num_failed_logins': np.random.randint(0, 5, size=num_samples),
    'root_shell': np.random.choice([0, 1], size=num_samples),
    'label': np.random.choice([0, 1, 2, 3, 4], size=num_samples)  # Labels as numbers (0 to 4)
}

# Create DataFrame
df = pd.DataFrame(data)

# Encode categorical features using LabelEncoder
label_encoder = LabelEncoder()
df['protocol_type'] = label_encoder.fit_transform(df['protocol_type'])
df['service'] = label_encoder.fit_transform(df['service'])
df['flag'] = label_encoder.fit_transform(df['flag'])

# Save the synthetic dataset to a CSV file
df.to_csv('synthetic_dataset.csv', index=False)

# Display the first few rows to ensure data generation is correct
print(df.head())


   duration  protocol_type  service  flag  src_bytes  dst_bytes  land  \
0        52              1        4     3       1265       2886     1   
1       496              0        0     2       1479       1183     0   
2       443              2        4     2       3784        502     0   
3       612              0        4     1        537       1552     0   
4       254              1        1     3       2570       3906     1   

   wrong_fragment  urgent  hot  num_failed_logins  root_shell  label  
0              32      24   86                  2           0      2  
1              90      78   51                  4           0      4  
2              79      33   34                  4           1      3  
3               5      81   55                  2           0      0  
4              23      19   74                  1           1      1  


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import os

# Load the dataset (replace with your actual dataset)
dataset = pd.read_csv('your_data.csv')

# Split the dataset into features (X) and target (y)
X = dataset.drop('label', axis=1)  # assuming 'label' is the target column
y = dataset['label']

# Initialize LabelEncoder for the target column
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Encode the target labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize LabelEncoders for categorical columns
protocol_type_encoder = LabelEncoder()
service_encoder = LabelEncoder()
flag_encoder = LabelEncoder()

# Encode categorical columns in the training set
X_train['protocol_type'] = protocol_type_encoder.fit_transform(X_train['protocol_type'])
X_train['service'] = service_encoder.fit_transform(X_train['service'])
X_train['flag'] = flag_encoder.fit_transform(X_train['flag'])

# Transform categorical columns in the testing set (without refitting)
X_test['protocol_type'] = protocol_type_encoder.transform(X_test['protocol_type'])
X_test['service'] = service_encoder.transform(X_test['service'])
X_test['flag'] = flag_encoder.transform(X_test['flag'])

# Initialize the RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')
print(classification_report(y_test, y_pred))

# Ensure the 'models' directory exists
if not os.path.exists('models'):
    os.makedirs('models')

# Save the trained model
joblib.dump(model, 'models/network_intrusion_model.pkl')

# Save the label encoders
joblib.dump(protocol_type_encoder, 'models/protocol_type_encoder.pkl')
joblib.dump(service_encoder, 'models/service_encoder.pkl')
joblib.dump(flag_encoder, 'models/flag_encoder.pkl')
joblib.dump(label_encoder, 'models/label_encoder.pkl')

# Optionally, save the processed data for later use
X_train.to_csv('processed_X_train.csv', index=False)
X_test.to_csv('processed_X_test.csv', index=False)
pd.DataFrame(y_train, columns=['label']).to_csv('processed_y_train.csv', index=False)
pd.DataFrame(y_test, columns=['label']).to_csv('processed_y_test.csv', index=False)


Accuracy: 19.93%
              precision    recall  f1-score   support

           0       0.20      0.23      0.22      6037
           1       0.21      0.20      0.20      5955
           2       0.20      0.20      0.20      6045
           3       0.20      0.19      0.19      5989
           4       0.19      0.17      0.18      5974

    accuracy                           0.20     30000
   macro avg       0.20      0.20      0.20     30000
weighted avg       0.20      0.20      0.20     30000

